In [ ]:
%matplotlib inline
from matplotlib import pylab as plt

### Make sure you have the geoserver VM running

this should show something like:

```
Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.
```

You can do this by going to the geoserver/ directory and running ```vagrant up```

In [ ]:
!cd ../geoserver && vagrant status

### Display geoserver status

This should ensure the client can successfully connect to your VM,  if you do not see the Geoserver 'Status' page then something is wrong!

In [ ]:
from IPython.core.display import display, HTML
from geonotebook.config import Config
geoserver = Config().vis_server
display(HTML(geoserver.c.get("/about/status").text))

### Get the rgb.tif file from data.kitware.com

In [ ]:
!wget -O /tmp/rgb.tif https://data.kitware.com/api/v1/file/57bf01ff8d777f10f269cff4/download

### Add the layer to the map

In [ ]:
DATA_DIR="/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/TOA/"

In [ ]:
from geonotebook.wrappers import RasterData
rd = RasterData(DATA_DIR +  'L57.Globe.month01.2010.hh09vv04.h6v1.doy003to027.TOA.v3.0.tiff')

# rdc = RasterCollection(["/tmp/rgb1.tif", "/tmp/rgb2.tif", "/tmp/rgb3.tif"])

In [ ]:
# gt.describe()

M.add_layer(rd.bands[1, 2, 3], opacity=0.9, range=(0,0.6))


In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(12, 12))
from skimage.exposure import rescale_intensity
# Display BRG
# n = M.layers['rgb'].region

# Flip the second axis to get RGB
n = np.apply_along_axis(np.flipud, 2, M.layers[-1].region)

ax.imshow(rescale_intensity(n), interpolation='none')

In [ ]:
import numpy as np

from skimage.color import rgb2gray
from skimage.exposure import rescale_intensity
from skimage import measure

def contours(level=0.5):
  fig, ax = plt.subplots(figsize=(16, 16))
  
  n = rescale_intensity(
    np.apply_along_axis(np.flipud, 2, M.layers[-1].region))

  contours = measure.find_contours(rgb2gray(n), level)


  ax.imshow(n, interpolation='none')

  ax.axis('tight')
  
  for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
  
  
contours(0.07)

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

interact(contours, level=(0.0, 1.0, 0.05))

In [ ]:
# Modified from
# http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_blob.html#sphx-glr-auto-examples-features-detection-plot-blob-py

from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
from skimage.color import rgb2gray


def blobs(max_sigma=30, num_sigma=10, threshold=.1):
  image = rescale_intensity(
        np.apply_along_axis(np.flipud, 2, M.layers[-1].region))
  image_gray = rgb2gray(image)


  blobs_log = blob_log(image_gray, 
                       max_sigma=max_sigma, 
                       num_sigma=num_sigma, 
                       threshold=threshold)
  # Compute radii in the 3rd column.
  blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

  blobs_list = [blobs_log]
  colors = ['yellow']
  titles = ['Laplacian of Gaussian']
  sequence = zip(blobs_list, colors, titles)

  fig, ax = plt.subplots(1, 1, figsize=(16, 16), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
  plt.tight_layout()

  # axes = axes.ravel()
  for blobs, color, title in sequence:
      ax.set_title(title)
      ax.imshow(image, interpolation='nearest')
      ax.set_axis_off()
      for blob in blobs:
          y, x, r = blob
          c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
          ax.add_patch(c)

  plt.show()
  
  
interact(blobs, max_sigma=(0, 60), num_sigma=(0, 20), threshold=(0.0, 0.2, 0.01))
